In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

In [3]:
qp = QuadraticProgram()

c = [2000, 4000, 8000, 15000]
t = 10
w = [20, 40, 70, 120]
d = 130

qp = QuadraticProgram()
qp.binary_var('x11')
qp.binary_var('x21')
qp.binary_var('x22')
qp.binary_var('x31')
qp.binary_var('x32')
qp.binary_var('x33')
qp.binary_var('x41')
qp.binary_var('x42')
qp.binary_var('x43')
qp.binary_var('x44')

qp.maximize(linear={'x11':c[0],
                    'x21':c[0], 'x22':c[1],
                    "x31":c[0], 'x32':c[1],'x33':c[2],
                    'x41':c[0], 'x42':c[1], 'x43':c[2], "x44":c[3]})

qp.linear_constraint(linear={'x11':1, 'x22':1, 'x33':1, 'x44':1}, sense='==', rhs=1, name='select_i')

qp.linear_constraint(linear={'x21':1, 'x22':-1}, sense='<=', rhs=0, name='x2cons')
qp.linear_constraint(linear={'x31':1, 'x32':1, 'x33':-2}, sense='<=', rhs=0, name='x3cons')
qp.linear_constraint(linear={'x41':1, 'x42':1, 'x43':1, 'x44':-3}, sense='<=', rhs=0, name='x4cons')

qp.linear_constraint(linear={'x11':w[0]+t,
                             'x21':t, 'x22':w[1]+t,
                             'x31':t, 'x32':t, 'x33':w[2]+t,
                             'x41':t, 'x42':t, 'x43':t, 'x44':w[3]+t}, sense='<=', rhs=d, name='deadline')

print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 2000 x11 + 2000 x21 + 4000 x22 + 2000 x31 + 4000 x32 + 8000 x33 + 2000 x41
      + 4000 x42 + 8000 x43 + 15000 x44
Subject To
 select_i: x11 + x22 + x33 + x44 = 1
 x2cons: x21 - x22 <= 0
 x3cons: x31 + x32 - 2 x33 <= 0
 x4cons: x41 + x42 + x43 - 3 x44 <= 0
 deadline: 30 x11 + 10 x21 + 50 x22 + 10 x31 + 10 x32 + 80 x33 + 10 x41 + 10 x42
           + 10 x43 + 130 x44 <= 130

Bounds
 0 <= x11 <= 1
 0 <= x21 <= 1
 0 <= x22 <= 1
 0 <= x31 <= 1
 0 <= x32 <= 1
 0 <= x33 <= 1
 0 <= x41 <= 1
 0 <= x42 <= 1
 0 <= x43 <= 1
 0 <= x44 <= 1

Binaries
 x11 x21 x22 x31 x32 x33 x41 x42 x43 x44
End



In [4]:
aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])

In [5]:
qaoa = MinimumEigenOptimizer(qaoa_mes)

In [6]:
qaoa_result = qaoa.solve(qp)
print(qaoa_result)

constraint select_i is infeasible due to substitution
constraint x4cons is infeasible due to substitution


optimal function value: 20000.0
optimal value: [1. 0. 0. 1. 0. 1. 0. 0. 1. 0.]
status: INFEASIBLE
